In [ ]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
# autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
autolib_dir = '/'.join(nb_dir.split('/')[:-1])
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

In [ ]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

In [ ]:
categorical_features = ['blabla']

In [ ]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count], 2, 2)#len(categorical_features))

features_to_gen = []

In [ ]:
len(features_to_gen)

In [ ]:
project_path = './' 

In [ ]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'project.log', file_loglevel=logging.INFO)

In [ ]:
def params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'svc':
        params = {
            'C': 0.911#np.random.rand(),
#             'metric': 'roc_auc'
        }
    elif model == 'logreg':
        params = {
            'penalty': np.random.choice(['l2']),
            'dual': np.random.choice([True, False]),
            'C': 1,#np.random.rand(),
            'metric': 'roc_auc'
        }
    elif model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 30, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc'
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': np.random.randint(2,10),
            'lr_init': 0.01,
            'lr_fin': np.random.randint(1,5)/1000,  # if == lr_init, then no lr decay
            'batch_size': np.random.choice([128, 256, 512, 1024]),
            "pred_batch_size": 5000,
            'max_ep': 100,
            'patience': 10, #np.random.randint(10, 25),
            'cat_emb_outdim': 30, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
                                  # then assige embed_outdim_dict to cat_emb_outdim
            'cat_emb_drop_rate': np.random.randint(1,4)/10, 
            'num_layers_dense_units': [1000, 500, 100],
            'num_layers_drop_rate': np.random.randint(2,6)/10,
            'combined_layers_dense_units': [100, 50],
            'combined_layers_drop_rate': np.random.randint(1,3)/10,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    elif model == 'stacknet_layer2_nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.01, # if == lr_init, then no lr decay
            'batch_size': 128,
            "pred_batch_size": 50000,
            'best_epoch': 1,
            'patience': 1,
            'categorical_feature': [],
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'num_layers_dense_units': [],
            'combined_layers_dense_units': [10, 5],
            'combined_layers_drop_rate': 0,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    return params, seed

In [ ]:
p, s = params_gen('stacknet_layer2_nn')
p, s

In [ ]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, params_gen)